In [2]:
# here i will try implrmrnting LSTM model #
# copied from https://analyticsindiamag.com/how-to-do-multivariate-time-series-forecasting-using-lstm/


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 

In [9]:
data = pd.read_csv('../input/metro-interstate-traffic-volume/Metro_Interstate_Traffic_Volume.csv')
print(data.shape)
data.head()

(48204, 9)


,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918


In [8]:
for i in data.select_dtypes('object').columns:
    le = LabelEncoder().fit(data[i])
    data[i] = le.transform(data[i]) 
X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()
X_data = X_scaler.fit_transform(data[['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main',
    'weather_description','traffic_volume']])
Y_data = Y_scaler.fit_transform(data[['traffic_volume']])
X_data.shape

(48204, 8)

In [10]:
def custom_ts_multi_data_prep(dataset, target, start, end, window, horizon):
    X = []
    y = []
    start = start + window
    if end is None:
        end = len(dataset) - horizon
    for i in range(start, end):
        indices = range(i-window, i)
        X.append(dataset[indices])
        indicey = range(i+1, i+1+horizon)
        y.append(target[indicey])
    return np.array(X), np.array(y) 

In [14]:
# As we are doing multiple-step forecasting, let’s allow the model to see past 48 hours of data
# and forecast the 10 hrs after data; for that, we set the horizon to 10.
hist_window = 48
horizon = 10
TRAIN_SPLIT = 30000
x_train, y_train = custom_ts_multi_data_prep(X_data, Y_data, 0, TRAIN_SPLIT, hist_window, horizon)
x_vali, y_vali = custom_ts_multi_data_prep(X_data, Y_data, TRAIN_SPLIT, None, hist_window, horizon)

print(x_train.shape)

(29952, 48, 8)


In [13]:
#The train window should contain eight variables and one target variable for about ten observations. 
# i use 48x8 matrix (48 obs, 8 vars) to predict next 10 values of y.

print ('Multiple window of past history\n')
print(x_train[0])
print ('\n Target horizon\n')
print (y_train[0]) 

Multiple window of past history

[[0.63636364 0.92972555 0.         0.         0.4        0.1
  0.64864865 0.76167582]
 [0.63636364 0.93320863 0.         0.         0.75       0.1
  0.05405405 0.62032967]
 [0.63636364 0.93391815 0.         0.         0.9        0.1
  0.51351351 0.65480769]
 [0.63636364 0.93569194 0.         0.         0.9        0.1
  0.51351351 0.69038462]
 [0.63636364 0.93894927 0.         0.         0.75       0.1
  0.05405405 0.67554945]
 [0.63636364 0.94081981 0.         0.         0.01       0.
  0.72972973 0.71167582]
 [0.63636364 0.94549618 0.         0.         0.01       0.
  0.72972973 0.76703297]
 [0.63636364 0.94772148 0.         0.         0.01       0.
  0.72972973 0.82623626]
 [0.63636364 0.9486245  0.         0.         0.2        0.1
  0.10810811 0.79546703]
 [0.63636364 0.94527042 0.         0.         0.2        0.1
  0.10810811 0.65521978]
 [0.63636364 0.93840101 0.         0.         0.2        0.1
  0.10810811 0.48612637]
 [0.63636364 0.93327313 